# Simple evolutionary algorithms

From [Wikipedia](https://en.wikipedia.org/wiki/Evolutionary_algorithm)):

> In artificial intelligence, an evolutionary algorithm (EA) is a subset of evolutionary computation,[1] a generic population-based metaheuristic optimization algorithm. An EA uses mechanisms inspired by biological evolution, such as reproduction, mutation, recombination, and selection. Candidate solutions to the optimization problem play the role of individuals in a population, and the fitness function determines the quality of the solutions (see also loss function). Evolution of the population then takes place after the repeated application of the above operators.

Evolutionary algorithms often work well since they make no assumptions about the "fitness landscape". They can be used in situations where gradient-based algorithms are not applicable because computing the gradient is not possible or impractical, and they often work surprisingly well in practice 

## The 1+1 algorithm

The 1+1 algorithm implements the simplest version of these rules, with only one parent being considered at each step (the first "1" in the name) and only one new offspring is generated (the second "1"). The child is generated by randomly mutating the parent and replaces the parent if is more fit. This process is repeated until a solution is found, or until a fixed "budget" of steps has been exhausted.

More precisely, assume that we have $n$ binary inputs and we want to optimize a function $f: \{0, 1\}^n \to \mathbb{R}$ of these inputs. The 1+1 algorithm can then be summarized as 

1. Generate a random bit string of length $n$. This the parent, $P$.
2. For a given number of times, do
    1. Generate the child, $C$, by flipping bits in the parent with probability $1/n$.
    2. If $f(C) \ge f(P)$, replace $P$ by $C$.
    
This is an example of a hillclimber: the algorithm only accepts solutions that are better than what it already has and never considers an inferior solution. This causes it to sometimes get stuck at near-optima, as we'll see below.

In [1]:
import numpy as np

def one_plus_one(f, parent, budget):
    n = len(parent)
    history = [parent]
    
    p = 1/n  # probability of mutating a bit in the parent
    for _ in range(budget):
        # produce the child by randomly mutating the parent
        mutate_mask = np.random.choice([0, 1], size=n, p=(1-p, p))
        child = parent ^ mutate_mask
        if f(child) >= f(parent):
            parent = child
        history.append(parent)        
    return np.asarray(history)

To print out the evolutionary history in a nice, compact way, we have the following routine. As is typical for functions that do a lot of formatting, it is more complicated than the actual algorithm. 

In [2]:
def print_one(index, bits, ndigits, nhistory):
    bitstr = "".join(str(b) for b in bits)
    if index % 10 not in (0, 5) and index != nhistory - 1:
        index = ""
    print("{:>{}} {}".format(index, ndigits, bitstr), end="\t")

def pretty_print(history, ncols=3):    
    row_len = 5
    while row_len * ncols < len(history):
        row_len += 5
    label_len = len(str(len(history)))
    for i in range(row_len):
        for j in range(ncols):
            idx = j*row_len + i
            if idx < len(history):
                print_one(idx, history[idx], label_len, history.shape[0])
        print()

This algorithm does reasonably well on simple problems. Let's take an example of the [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem) where we try to fill a knapsack with items on display. Each item has a weight and a price, and we want to fill our knapsack with the most valuable items. However, our knapsack can only hold a fixed weight, so we'll have to make tradeoffs.

In [3]:
# Five items
weights = np.array([1, 1, 2, 4, 12])
prices = np.array([1, 2, 2, 10, 4])

def knapsack(included):
    total_weight = np.dot(weights, included)
    if total_weight > 15:
        return -float("inf")
    return np.dot(prices, included)

In [4]:
initial = np.zeros(5, dtype=int)
history = one_plus_one(knapsack, initial, budget=50)

We see that in most cases we obtain the optimal solution (include every single item except for the heaviest item, which corresponds to the bit vector $11110$) in 20-30 steps. This is not very spectacular, given that the search space contains only $2^5 = 32$ combinations to try. Depending on the initial conditions, some solutions also take significantly more than 30 steps, so that the 1+1-algorithm does strictly worse than the strict enumeration of all possibilities.

In [5]:
pretty_print(history)

 0 00000	20 11110	40 11110	
   00000	   11110	   11110	
   00000	   11110	   11110	
   00000	   11110	   11110	
   01100	   11110	   11110	
 5 01110	25 11110	45 11110	
   01110	   11110	   11110	
   01110	   11110	   11110	
   01110	   11110	   11110	
   01110	   11110	   11110	
10 01110	30 11110	50 11110	
   01110	   11110	
   01110	   11110	
   01110	   11110	
   01110	   11110	
15 01110	35 11110	
   11110	   11110	
   11110	   11110	
   11110	   11110	
   11110	   11110	


## The trap function: a pathological example

There are certain objective functions that lead the 1+1 algorithm astray, and the trap function is one of them. The trap function is defined as
$$
    \text{trap}(x) = \sum_{i=1}^n x_i + (n + 1) \prod_{i=1}^n (1-x_i),
$$
where $n$ is the number of inputs. The global optimum is $x_0 = (0,\ldots, 0)$, and the expected time to find this optimum is $O(n^n)$, which is astonishing given that we're talking about a simple, linear function.

What makes the trap function so ill-suited for the 1+1 algorithm? For inputs that have at least one bit set to 1, the product term vanishes, and the trap function counts the number of 1s in the input. More 1s results in higher fitness, so flipping bits at random will quickly drive the input towards the all-1 vector, $x_1 = (1, \ldots, 1)$. But this is not the global optimum, since $f(x_1) = n$ but $f(x_0) = n+1$. Getting from $x_1$ to $x_0$ requires flipping all bits at once, which happens with probability $1/n^n$. As a result, we have to wait an expected $n^n$ steps to reach $x_0$, and $x_1$ effectively traps the algorithm.

In [6]:
def trap(x):
    return np.sum(x) + (len(x) + 1)*np.prod(1 - x)

In [7]:
x = np.array([0, 1, 0, 1])
history = one_plus_one(trap, x, 1000)

pretty_print(history, ncols=7)

   0 0101	 145 1111	 290 1111	 435 0000	 580 0000	 725 0000	 870 0000	
     0101	     1111	     1111	     0000	     0000	     0000	     0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
   5 1111	 150 1111	 295 1111	 440 0000	 585 0000	 730 0000	 875 0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
  10 1111	 155 1111	 300 1111	 445 0000	 590 0000	 735 0000	 880 0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
     1111	     1111	     1111	     0000	     0000	     0000	     0000	
     1

## Getting unstuck: Luby's restart strategies

Looking at the output of the above run, it is clear that we are not getting very far once the state $111...111$ shows up. Without modifying the algorithm, what can we do once we realize we are stuck away from the global optimum? One option is to periodically restart with a different initial condition, run for a while, and see if we get anywhere nearer. How long each run is supposed to take is up to you, but there are different strategies that are known to be optimal:

1. _Luby's uniform strategy_: Let the algorithm run with a fixed budget $B$. If convergence has not been attained, select another initial condition and run again with budget $B$. Repeat until convergence.
2. _Luby's universal strategy_: Instead of repeating with a fixed budget, use budgets $(1, 1, 2, 1, 1, 2, 4, 1, \ldots)$ ([OEIS A182105](https://oeis.org/A182105)).

These strategies are known to be optimal for _Las Vegas algorithms_: algorithms for which there is a way of deciding convergence, but whose runtime is a random variable. Luby's uniform strategy is useful when the runtime distribution is known, since that allows you to estimate an optimal value for $B$. If that is not the case, Luby's universal strategy empirically samples from the runtime distribution by probing successively longer runtimes.

For the trap function, Luby's uniform strategy results in run time $O(2^n)$, since new guesses are generated by uniform sampling (and the probability of selecting a particular sequence is $2^{-n}$) and each guess runs for $B$ steps. 

# References

- Stefan Droste, Thomas Jansen, Ingo Wegener, On the analysis of the (1+1) evolutionary algorithm, Theoretical Computer Science, Volume 276, Issues 1–2, 2002, Pages 51-81, https://doi.org/10.1016/S0304-3975(01)00182-7.

- Michael Luby, Alistair Sinclair, David Zuckerman, Optimal Speedup of Las Vegas Algorithms, 1993.

- Matteo Gagliolo and Jurgen Schmidhuber. Learning Restart Strategies, 2007.

- Friedrich, T., Kötzing, T., Quinzan, F., & Sutton, A. M. (2018). Improving the run time of the (1 + 1) evolutionary algorithm with luby sequences. Proceedings of the Genetic and Evolutionary Computation Conference on - GECCO ’18. doi:10.1145/3205455.3205525 